In [1]:
#Librería necesaria
#! pip install cloud-sql-python-connector["pg8000"]
#Importación de librerías
import pandas as pd
import numpy as np
import pg8000
import sqlalchemy
from google.cloud import bigquery   
from google.cloud.sql.connector import Connector

In [2]:
client= bigquery.Client()

In [3]:
#Función crea conexión a Postgres
def create_connection():
    POSTGRES_CONNECTION_NAME = "conexion-datos-rdf:us-central1:rdf-db"
    POSTGRES_USER = "postgres"
    POSTGRES_PASS = "n3FLOvsqa8nqtjym"
    POSTGRES_DB = "rdf-db"

    connector = Connector()

    def init_connection_engine() -> sqlalchemy.engine.Engine:
        def getconn() -> pg8000.dbapi.Connection:
            conn: pg8000.dbapi.Connection = connector.connect(
                POSTGRES_CONNECTION_NAME,
                "pg8000",
                user=POSTGRES_USER,
                password=POSTGRES_PASS,
                db=POSTGRES_DB,
            )
            return conn

        engine = sqlalchemy.create_engine(
            "postgresql+pg8000://",
            creator=getconn,
        )
        engine.dialect.description_encoding = None
        return engine

    db = init_connection_engine()

    return db

In [94]:
#SOLO SI ES NECESARIO REEMPLAZAR LOS DATOS EN TABLA POSTGRES
#Se cargan datos desde archivo excel a tabla diccionarios.user_tags

# data = pd.read_excel('C:/Users/alopez/Desktop/tags_tabla.xlsx')

# schema = "diccionarios"
# table = "user_tags"

# chunk_size = 32767 // len(data.columns)
# chunks = (len(data) // chunk_size) + 1
# print(f"Se subirán {chunks} chunks de con un máximo de {chunk_size} filas")

# connection = create_connection()

# connection.execute('''TRUNCATE TABLE {0}.{1}'''.format(schema, table))
# data.to_sql(table, connection, schema=schema, if_exists='append', method='multi', index=False, chunksize=chunk_size)
# print("Datos cargados exitosamente a la ficha de los usuarios")

In [ ]:
#Consulta tabla user_tags desde Postgres
connection = create_connection()
df_deftags = pd.read_sql_query("SELECT * FROM diccionarios.user_tags", con=connection)

In [10]:
df_querys = df_deftags['table_query'].unique()
df_querys[0]

'SELECT email, genero, redad_ipsos, region, consumo_correos, consumo_live, consumo_od, consumo, desktop, mobile, apps, cia FROM `conexion-datos-rdf.usuarios.ficha_rdf`'

In [11]:
#CREAR UN CICLO PARA CUALQUIER QUERY
tags_dict ={}
# Primero se recorre longitud de df_querys, que contiene las distintas consultas encontrada en la tabla user_tags 
# almacenada en Postgres SQL
for i in range(len(df_querys)):   
    query = df_querys[i]
    # Se divide dataframe que contiene información de user_tags de acuerdo a las filas donde el campo table_query
    # es igual al query
    df_tag_query = df_deftags[df_deftags['table_query'] == query]
    # Se consulta query en BigQuery para traer la información de la query
    df_query= client.query(query).to_dataframe(create_bqstorage_client=False)

    # Por cada fila de user_tags, se define nombre del tag, key y clave
    # Cuando la clave corresponde a un booleano o una lista se realizan las conversiones necesarias
    for idx, row in df_tag_query.iterrows():
        name_tag = row["tag"]
        key_tag = row["table_key"]
        value_tag = row["table_value"]
        if row["table_key"] == "_id":
            value_tag = value_tag.split(',')
            tags_dict[name_tag] = list(df_query[df_query[key_tag].isin(value_tag)]["email"])
        elif row["table_value"] == 'True':
            tags_dict[name_tag] = list(df_query[df_query[key_tag] == bool(True)]["email"])
        elif row["table_value"] == 'False':
            tags_dict[name_tag] = list(df_query[df_query[key_tag] == bool(False)]["email"])
        elif row["table_value"] == None:
            tags_dict[name_tag] = list(df_query[df_query[key_tag] == None]["email"])
        else:
            tags_dict[name_tag] = list(df_query[df_query[key_tag] == (value_tag)]["email"])
        

In [13]:
tags_dict

{'g_mujer': ['osita-8998@hotmail.com',
  'a.esterio@hotmail.com',
  'alee.da7@gmail.com',
  'alevasquez@gmail.com',
  'alejandracortes81@gmail.com',
  'alesepv@gmail.com',
  'ale.labarca.g@gmail.com',
  'am.cuellar@hotmail.com',
  'aaaraya1992@gmail.com',
  'cortezmuoz.a@gmail.com',
  'anaspikinhayes@gmail.com',
  'aluisa.neira@gmmail.com',
  'anita.laborde@gmail.com',
  'arojasgoic@outlook.com',
  'andreveliz00@gmail.com',
  'angelakine@hotmail.com',
  'angelamvergaraz@gmail.com',
  'ange_cornejo@hotmail.com',
  'a-ang-acuadrao-o@hotmail.com',
  'anita.lira0591@gmail.com',
  'futbariel@hotmail.com',
  'arlette.lobosc@yahoo.cl',
  'beatrizcollantes20@gmail.com',
  'mcruces.trebulco@gmail.com',
  'brenda.patricia.ceron.arancibia@gmail.com',
  'camilapmejiasc@gmail.com',
  'camila.mezabarahona@gmail.com',
  'camilabarahona@live.cl',
  'monse371@gmail.com',
  'yo_o_mmmm@hotmail.com',
  'cmellasa@gmail.com',
  'c.gloriariquelme@gmail.com',
  'cgparodi@yahoo.com',
  'carolita.dg@gmail.com',